# Imports

In [353]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist


# Input

In [354]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [355]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [356]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [357]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [358]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[1.04127364 1.04187946 1.04127364 ... 0.96939069 1.40714958 0.96939069]
[0.8303662  0.83084931 0.8303662  ... 0.77304297 1.12213486 0.77304296]
[0.66966854 0.66975949 0.66966854 ... 0.65878875 0.72211903 0.65878875]
(5999,)


### Multinomial Sampling

In [359]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 1 0]


In [360]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[81.27219  ,  5.7970243, 95.28757  ],
       [76.61191  ,  5.7970243, 96.536285 ],
       [85.85287  ,  4.7321377, 93.253334 ],
       [89.788124 ,  5.2797017, 86.876625 ],
       [84.968575 ,  5.2797017, 93.51016  ]], dtype=float32)

# KNN extended graph

In [361]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [362]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11735


# Edge Predictor

In [363]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [364]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [365]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [366]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [367]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 7)	1
  (1, 8)	1
  (1, 11)	1
  (1, 12)	1
  (1, 13)	1
  (1, 14)	1
  :	:
  (1498, 1382)	1
  (1498, 1444)	1
  (1498, 1445)	1
  (1498, 1446)	1
  (1498, 1477)	1
  (1498, 1478)	1
  (1498, 1479)	1
  (1498, 1481)	1
  (1498, 1490)	1
  (1498, 1495)	1
  (1498, 1496)	1
  (1499, 1432)	1
  (1499, 1437)	1
  (1499, 1438)	1
  (1499, 1439)	1
  (1499, 1451)	1
  (1499, 1466)	1
  (1499, 1467)	1
  (1499, 1468)	1
  (1499, 1469)	1
  (1499, 1474)	1
  (1499, 1475)	1
  (1499, 1476)	1
  (1499, 1483)	1
  (1499, 1484)	1
[[7.17492767e-02 7.17703184e-02 7.17492768e-02 ... 7.18050255e-02
  7.18050254e-02 7.38170614e-02]
 [1.00929269e+00 5.98517303e-02 5.97873916e-02 ... 5.99581600e-02
  5.99581598e-02 6.69591673e-02]
 [1.12913959e+00 6.66881732e-01 6.70630129e-02 ... 6.71325237e-02
  6.71325236e-02 6.98

In [368]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [369]:
print(A_s)  # symmétrique
print(A_s.shape)

[[3.53206797e-16 3.01755575e-16 3.36028533e-16 ... 9.13188228e-15
  3.44038604e-15 7.01188158e-10]
 [3.01755575e-16 2.57874894e-16 2.88636684e-16 ... 7.89200287e-15
  2.93084472e-15 6.32555179e-10]
 [3.36028533e-16 2.88636684e-16 3.23278711e-16 ... 8.65271459e-15
  3.30086325e-15 6.62552119e-10]
 ...
 [9.13188228e-15 7.89200287e-15 8.65271459e-15 ... 3.57969080e-13
  7.71007863e-14 5.45711300e-08]
 [3.44038604e-15 2.93084472e-15 3.30086325e-15 ... 7.71007863e-14
  6.51088542e-14 2.27960547e-09]
 [7.01188158e-10 6.32555179e-10 6.62552119e-10 ... 5.45711300e-08
  2.27960547e-09 2.56155600e-02]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [370]:
nodes = list(extended_graph.nodes())
edges = list(extended_graph.edges())

igraph_g = igraphGraph(directed=extended_graph.is_directed())

igraph_g.add_vertices(nodes)
igraph_g.add_edges([(nodes.index(u), nodes.index(v)) for u, v in edges])
print(igraph_g.summary())

IGRAPH UN-- 1500 11735 -- 
+ attr: name (v)


In [371]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[709 779 780]
 [779 780 951]
 [896 929 930]]
32095


In [372]:
triangles = np.array(igraph_g.vs['name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[75.         6.0434117  1.3966535]
  [77.26702    6.9390492  1.2822151]
  [77.517784   6.9390492  1.5329764]]

 [[77.26702    6.9390492  1.2822151]
  [77.517784   6.9390492  1.5329764]
  [78.63775    7.6759295  1.1570231]]

 [[77.49305   -6.5098743 82.63936  ]
  [77.13951   -7.328478  84.49455  ]
  [77.44897   -7.6759295 84.971085 ]]]
32095


In [373]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(triangles))
p_init = (A_s_ij + A_s_ik + A_s_jk) / 3

print(p_init.shape)
print(p_init)

(32095,)
[5.20771767e-11 5.54085308e-13 9.13701344e-13 ... 2.79686702e-09
 5.62950509e-10 1.35932848e-09]


#### Calculate barycenter

In [374]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32095, 3)
[[76.59494    6.6405034  1.4039484]
 [77.80752    7.1846757  1.3240715]
 [77.36051   -7.1714272 84.034996 ]]


#### KNN Tri

In [375]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [376]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32095, 19, 3)


#### calculate e norm matrix

In [377]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32095, 3)
[[2.4402146 2.675815  0.3546312]
 [0.3546312 1.5612686 1.3923557]]


#### Calculate r

In [378]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32095,)

In [379]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32095, 19)
(32095, 19)


In [380]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32095, 19, 5)


#### Calculate f

In [381]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

1.0000000000000269
(32095,)
[3.11240170e-05 3.11240170e-05 3.11240170e-05 ... 3.11240171e-05
 3.11240170e-05 3.11240170e-05]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_5324\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [382]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[51.         6.1633806 83.71886  ]
 [39.         7.0402236 84.7203   ]
 [39.         8.642877  88.3445   ]]


In [383]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 53
Number of edges: 243


In [384]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [399]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [400]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[ 81.27219     5.7970243  95.28757  ]
 [ 81.08616     6.2811418  94.86973  ]
 [ 82.00088     5.7970243  94.94     ]
 ...
 [ 81.9768     -7.8332825  91.94333  ]
 [ 80.21027    -7.8332825  92.90247  ]
 [ 39.          6.620764  -10.797024 ]]
dim sample points :  1500


Distance Matrix

In [401]:
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between barycenters b et b_hat after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='sqeuclidean').T

distances_matrix = matrix_distance_euclidienne_barycentres(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 8450.76443414  6254.81262802  8442.92076056 ... 11581.86874712
  12322.65395005 11452.12147934]
 [ 8378.68689456  6193.35034174  8370.18945292 ... 11499.50004762
  12236.48853529 11370.42072523]
 [ 8396.92477568  6209.95619506  8389.11782796 ... 11555.54853858
  12314.27702495 11426.25596782]
 ...
 [ 7868.94779405  5763.82288588  7879.28246918 ... 10746.20730956
  11504.86271657 10617.4195403 ]
 [ 8017.51240745  5887.60264417  8027.75805009 ... 10830.57146747
  11545.57547952 10700.5289097 ]
 [ 1563.97824874  2103.80387876  1552.90761466 ...   389.44127016
    254.68421461   398.68421461]]


In [402]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [403]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [404]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00020521 0.00020524 0.00020521 ... 0.00020188 0.00022129 0.00020188]


In [406]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 2.6621353009661055


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [407]:
# Creation igraph
nodes_original = list(graph.nodes())
edges_original = list(graph.edges())
igraph_g_original = igraphGraph(directed=extended_graph.is_directed())
igraph_g_original.add_vertices(nodes_original)
igraph_g_original.add_edges([(nodes_original.index(u), nodes_original.index(v)) for u, v in edges_original])
print(igraph_g_original.summary())

#Find triangles
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['name'])[triangles_ids_igraph_original]
print("triangles shape : ", triangles.shape)
print(triangles[0])

# Calculate barycenters
b = np.mean(triangles, axis=1)
print("shape b : ", b.shape)
print("Barycentre b: ", b)

IGRAPH UN-- 5999 17991 -- 
+ attr: name (v)
triangles shape :  (11994, 3, 3)
[[39.        8.793697 -4.666995]
 [51.        8.793697 -4.666995]
 [39.        8.793697 -5.333005]]
shape b :  (11994, 3)
Barycentre b:  [[43.          8.793697   -4.8889985 ]
 [55.          1.0515273   3.7313068 ]
 [55.         -8.515905   -2.804371  ]
 ...
 [75.05613    -0.94137865 21.        ]
 [75.01313    -0.3882971   8.099999  ]
 [75.03516    -0.72036856 25.300001  ]]


In [408]:
# Get finals selected triangles & calculate barycenters
b_hat = np.mean(selected_triangles, axis=1)
print(b_hat.shape)
print("B_HAT : ", b_hat)

(500, 3)
B_HAT :  [[ 4.3000000e+01  7.2821603e+00  8.5594543e+01]
 [ 5.1000000e+01  6.7756653e+00  8.4895058e+01]
 [ 3.9000000e+01  2.5407698e+00 -7.8802980e-02]
 ...
 [ 3.9000000e+01  5.4586381e-01 -1.1749581e+00]
 [ 3.9000000e+01  5.5426472e-01 -1.3968025e+00]
 [ 3.9000000e+01  2.8539190e+00  8.8019569e+01]]


test matrice des normes euclidiennes entre b et b_hat

In [394]:
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print(distances.shape)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


(500, 11994)
Distances euclidiennes entre les barycentres b et b_hat :
 [[8189.55611095 6884.41030522 8207.94697681 ... 5267.67707271
  7089.08077535 4725.72353659]
 [8129.24919878 6636.32021412 7941.02194463 ... 4720.8285272
  6525.43375029 4185.45002267]
 [  78.23708386  272.73477986  385.67878806 ... 1756.48576436
  1372.41778783 1953.2512083 ]
 ...
 [  97.82085435  280.32713081  340.77064756 ... 1793.98512863
  1383.84303988 2001.0592979 ]
 [  96.08368276  282.54477539  340.24923502 ... 1803.89818465
  1388.02321645 2012.87650319]
 [8683.28295682 7363.75982917 8634.26107787 ... 5806.07143797
  7694.59517549 5245.05225955]]


In [410]:
minimums_b = np.min(distances, axis=1)
print(minimums_b.shape)
print(minimums_b[:5])

(500,)
[ 0.13395437 16.13610641  1.40520565 24.98210947 16.52787497]


In [414]:
d_f_S_Ss = np.sum(selected_triangles_indexes * minimums_b)
print(d_f_S_Ss)

42164893.50211412


matrice sqeuclidean b

In [396]:
def normes_euclidiennes_carrees_b(bar_b):
    # Calculer la matrice des normes euclidiennes au carré
    normes_carrees = np.sum(bar_b**2, axis=1)

    return normes_carrees

matrix_b = normes_euclidiennes_carrees_b(b)
print(matrix_b.shape)
print(matrix_b)

(11994,)
[1950.2314 3040.0283 3105.3853 ... 6075.3086 5692.7305 6270.884 ]


In [397]:
def matrix_distances_after_knn(b_hat_n, number_neigh=3):
    """
    Calculate distances between barycenters after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """

    # Connect the extended graph using the k-NN process
    extended_graph = knn_and_extended_graph(b_hat_n, number_neigh)

    # Extract barycenters after the k-NN process
    b_hat_extended = np.array(list(extended_graph._node.keys()))

    # Calculate distances after k-NN
    distances_after = cdist(b_hat_extended, b_hat_extended)

    return distances_after

# Example usage with b_hat
distances_after = matrix_distances_after_knn(b_hat)

# Print the results
print("Distances After k-NN:\n", distances_after)

Distances After k-NN:
 [[ 0.          0.22200068  0.4237945  ... 14.54013091 14.90515569
  14.57769275]
 [ 0.22200068  0.          0.22199403 ... 14.51659334 14.8849859
  14.63643622]
 [ 0.4237945   0.22199403  0.         ... 14.6207881  14.9922894
  14.80831344]
 ...
 [14.54013091 14.51659334 14.6207881  ...  0.          0.44368622
   6.72768351]
 [14.90515569 14.8849859  14.9922894  ...  0.44368622  0.
   6.57809029]
 [14.57769275 14.63643622 14.80831344 ...  6.72768351  6.57809029
   0.        ]]


In [398]:
def probabilistic_surface_distance_extended(b_i, b_hat_i, p_b_hat_i):
    """
    Calculate Probabilistic Surfaces Distance 

    Parameters:
    - b: array-like, shape (N, 3)
        Barycenters of the ground truth surface.
    - b_hat: array-like, shape (M, 3)
        Barycenters of the generated triangles.
    - p_b_hat: array-like, shape (M,)
        Probabilities corresponding to the generated triangles.

    Returns:
    - d_S_Ss: float
        Probabilistic Surfaces Distance.
    """
    # Debugging print statements
    print("b shape:", b_i.shape)
    print("b_hat shape:", b_hat_i.shape)
    print("p_b_hat shape:", p_b_hat_i.shape)

    p_b_hat_i = normalized_inclusion_score.reshape((-1, 1))
    dist_matrixs_1= distances_after - matrix_b
    #dist_matrixs_2= matrix_b - distances_after

    d_S_Ss = np.sum(p_b_hat_i * np.min(np.sum(dist_matrixs_1)**2))

    

    return d_S_Ss

result_distance = probabilistic_surface_distance_extended(b, b_hat, normalized_inclusion_score)
print("Probabilistic Surfaces Distance:", result_distance)

b shape: (11994, 3)
b_hat shape: (500, 3)
p_b_hat shape: (5999,)


ValueError: operands could not be broadcast together with shapes (497,497) (11994,) 

## Triangle Collision Loss

In [ ]:
def triangle_collision_loss(generated_triangles, probabilities):
    """
    Calculate triangle collision loss to penalize triangles that penetrate each other.

    Parameters:
    - generated_triangles (numpy.ndarray): Array representing the generated triangles.
    - probabilities (numpy.ndarray): Array of probabilities for each generated triangle.

    Returns:
    - float: Triangle collision loss.
    """
    num_triangles = len(generated_triangles)

    # Create a 3D array to represent all combinations of triangle pairs
    triangle_pairs_i, triangle_pairs_j = np.meshgrid(range(num_triangles), range(num_triangles), indexing='ij')

    # Filter out pairs where i is not equal to j
    valid_pairs_mask = triangle_pairs_i != triangle_pairs_j

    # Extract triangles and probabilities for each pair
    triangles_i = generated_triangles[triangle_pairs_i[valid_pairs_mask]]
    triangles_j = generated_triangles[triangle_pairs_j[valid_pairs_mask]]
    probs_i = probabilities[triangle_pairs_i[valid_pairs_mask]]

    # Calculate the plane normals and distances for all triangles
    plane_normals = np.cross(triangles_i[:, 1] - triangles_i[:, 0], triangles_i[:, 2] - triangles_i[:, 0])

    # Calculate the edge vectors for all edges of triangles_j
    edge_vectors = triangles_j[:, [1, 2, 0], :] - triangles_j[:, [0, 1, 2], :]

    # Calculate the dot product of edge vectors and plane normals
    dot_products = np.sum(np.cross(edge_vectors, plane_normals[:, np.newaxis, :]) * triangles_i[:, 0], axis=2)

    # Count the number of faces penetrated for each triangle
    faces_penetrated = np.sum(dot_products < 0, axis=1)

    # Calculate the loss for each triangle
    loss_per_triangle = probs_i * faces_penetrated

    # Sum up the losses and normalize by the number of triangles
    triangle_losses = np.sum(loss_per_triangle) / num_triangles

    return triangle_losses

